In [17]:
%matplotlib inline
from time import time

from math import sqrt, floor
import numpy as np

import matplotlib.pyplot as plt
plt.style.use = "default"

import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

from IPython.core.debugger import set_trace

import sys
sys.path.append('../../')
from rfc import RandomForest
from Experiment import Experiment

In [18]:
def merge(df, labels):
    return labels.merge(df, left_index=True,right_index=True)

In [19]:
def mprint(*args):
    for arg in args:
        print(arg)
        print(" ")

In [20]:
train = pd.read_csv("cleaned_train.csv")
labels = pd.read_csv("cleaned_labels.csv")
test = pd.read_csv("cleaned_test.csv", header=None)

In [21]:
train = train.drop(train.columns[0], axis=1)
test = test.drop(test.columns[0], axis=1)

labels = labels.drop(labels.columns[0], axis=1) 
labels = labels.rename(columns={"5":"label"})

In [22]:
train.describe()

,0,1,2,3,4,5,6,7,8,9,10
count,1118.000,1118.000,1118.000,1118.000,1118.000,1118.000,1118.000,1118.000,1118.000,1118.000,1118.000
mean,8.306,0.528,0.267,2.558,0.087,15.906,46.772,0.997,3.309,0.653,10.404
std,1.725,0.179,0.192,1.458,0.045,10.468,32.554,0.002,0.153,0.164,1.057
min,4.600,0.120,0.000,0.900,0.012,1.000,6.000,0.990,2.870,0.330,8.700
25%,7.100,0.390,0.100,1.900,0.071,7.000,22.000,0.996,3.200,0.550,9.500
50%,7.900,0.520,0.250,2.200,0.080,14.000,38.000,0.997,3.300,0.620,10.100
75%,9.200,0.639,0.420,2.600,0.091,21.750,63.000,0.998,3.400,0.720,11.000
max,15.600,1.580,0.790,15.500,0.611,72.000,165.000,1.004,4.010,1.980,14.000


In [23]:
labels["label"].value_counts()

5    470
6    447
7    140
4     39
8     14
3      7
Name: label, dtype: int64

In [24]:
m = merge(train, labels).groupby("label")

In [25]:
m.count().apply(lambda x: x / train.shape[0]).T.mean()

label
3   0.006
4   0.035
5   0.420
6   0.400
7   0.125
8   0.013
dtype: float64

In [26]:
s_rows = 120
row_indices = np.random.choice(train.shape[0], size=s_rows, replace=False)
merge(train.iloc[row_indices], labels).groupby("label").count().apply(lambda x: x / s_rows).T.mean()

label
4   0.008
5   0.433
6   0.367
7   0.175
8   0.017
dtype: float64

In [27]:
init_config_names = ["num_trees", "num_features", "num_sample_rows", "max_tree_depth", "min_split_samples", "bias_class", "bias_amount"]
init_config_values = [   10,          None,           150,              20,                2,                 None,            0]

e = Experiment(train, labels, RandomForest, init_config_names, init_config_values, verbose=False)

In [28]:
e.run_trial()

Running trail #1
------------------------------------
 
*************************
 
Running fold 1 of 10
 
*************************
 


KeyError: -1

In [ ]:
perf, train_test = e.trial_results()

In [ ]:
perf

In [ ]:
e.experiment_data[0][2]

In [ ]:
e.tweak("num_trees", 20).tweak("num_features", 11).tweak("num_sample_rows", 120)

In [ ]:
r = e.run_trial()

In [ ]:
t = []
for c, s in test.iterrows():
    t.append(e.experiment_data[-1][0].predict(s))

pd.Series(t).to_csv("predictions5.txt", sep="\t", index=False, header=False)